In [17]:
import cv2, datetime, pandas

In [36]:
first_frame = None #'first_frame' variable will store 1st frame from 'video', will be used to compare subsequent frames.

status_list = [None] #will used detect object appearance and setting (Start & End) timings (initializing with None element, as we need atleast 2 elements to execute further)

times = [] #a list of Start & End timings of object appearance, used to populate pandas dataframe

data_frame = pandas.DataFrame(columns=['Start', 'End']) #pandas dataframe will be used to store Start & End time of object/s appeared in video

video = cv2.VideoCapture(0) #captures video from built-in camera. (pass arg. '1' for webcam & so on)

#while loop will execute frame by frame
while True:    
    check, frame = video.read() #video.read() will return 2 values (a bool if gets frame {check} & a 3d array of image {frame})
    
    status = 0 #used to identify object's appearance/presence
    
    gray_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) #converting frames to grayscale
    
    blured_img = cv2.GaussianBlur(gray_img, (21,21), 0 ) #converting grayscale image to blured
    
    #saving the 1st frame for comparision
    if first_frame is None:
        first_frame = blured_img
        
    delta_frame = cv2.absdiff(first_frame, blured_img) #comparing & finding difference b/w 1st frame & current frame
    
    #provide a threshold value, if pixel difference > 30 then convert it to white {255}, else black
    thresh_delta = cv2.threshold(delta_frame, 30, 255, cv2.THRESH_BINARY)[1]
    thresh_delta = cv2.dilate(thresh_delta, None, iterations=0 )
    
    #finding contours/outlines/borders of detected object
    (contours,_) = cv2.findContours(thresh_delta.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    #eliminating small objects (fly, noise) from making rectangle on them
    for contour in contours:
        if cv2.contourArea(contour) < 1000:
            continue #will break the 'for loop'
        
        status = 1 #sets status to 1 (means object is detected)
        
        #getting x, y cooedinates, width & height of contours (outlines) for making rectangle on them
        (x, y, w, h) = cv2.boundingRect(contour)
        
        #making rectangles (x, y coordinates starts from top-left (0,0)), giving green color {0,255,0} & 3 channels (RGB)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0,255,0), 3)
        
    status_list.append(status) #appending status_list (e.g:[None, 0]), appends 1 if object detected, else 0
    
    #comment this line to see whole video status_list
    status_list = status_list[-2:] #shorten the list to 2 last elements (we need only 2nd last & last element to detect object appearance)
     
    #it appends 'times' list with current time, if 2nd last frame detected NO object and last frame detects object, will be used as detection Start Time
    if status_list[-2] == 0 and status_list[-1] == 1:
        times.append(datetime.datetime.now())
        
    #it appends 'times' list with current time, if 2nd last frame detectes object and last frame detects NO object, will be used as detection End Time
    if status_list[-2] == 1 and status_list[-1] == 0:
        times.append(datetime.datetime.now())
        
    #displaying 'frame' = original video + rectangles), 'Capturing' = grayscale video, 'delta' = differences & 'thresh' = displaying differences in Black and White
    cv2.imshow('frame', frame)
    cv2.imshow('Capturing', gray_img)
    cv2.imshow('delta', delta_frame)
    cv2.imshow('thresh', thresh_delta)
    
    #frame will change after 1 millisecond
    key = cv2.waitKey(1)
    
    #will break the while loop on pressing 'q'
    if key == ord('q'):
        break
        
print(status_list)
print(times)

#store Start & End timings from 'times' list to Pandas dataframe
for i in range(0, len(times)-1, 2): #if same length of 'times' pass then error will occur, if having moving object in last frame of video, so -1 
    #must use same names (i.e: data_frame = data_frame.append()), like: a = a + 1 (otherwise only last values will append)
    data_frame = data_frame.append({'Start': times[i], 'End': times[i+1]}, ignore_index = True)

print(data_frame)
#saving dataframe to a .csv file
data_frame.to_csv('times.csv')

video.release()

#will close all windows (executes after while loop)
cv2.destroyAllWindows()

[1, 1]
[datetime.datetime(2020, 3, 14, 22, 21, 14, 488609), datetime.datetime(2020, 3, 14, 22, 21, 17, 922205), datetime.datetime(2020, 3, 14, 22, 21, 18, 262155), datetime.datetime(2020, 3, 14, 22, 21, 19, 12968), datetime.datetime(2020, 3, 14, 22, 21, 20, 657284), datetime.datetime(2020, 3, 14, 22, 21, 27, 989519), datetime.datetime(2020, 3, 14, 22, 21, 28, 116887), datetime.datetime(2020, 3, 14, 22, 21, 29, 587762), datetime.datetime(2020, 3, 14, 22, 21, 29, 671999), datetime.datetime(2020, 3, 14, 22, 21, 30, 122620), datetime.datetime(2020, 3, 14, 22, 21, 30, 457776), datetime.datetime(2020, 3, 14, 22, 21, 30, 790540), datetime.datetime(2020, 3, 14, 22, 21, 30, 917927), datetime.datetime(2020, 3, 14, 22, 21, 51, 650778), datetime.datetime(2020, 3, 14, 22, 21, 51, 782715)]
                       Start                        End
0 2020-03-14 22:21:14.488609 2020-03-14 22:21:17.922205
1 2020-03-14 22:21:18.262155 2020-03-14 22:21:19.012968
2 2020-03-14 22:21:20.657284 2020-03-14 22:21